In [1]:
import sys
#Indicate operating environment and import core modules
location_input = input("what computer are you on? a = Bens, b = gpucluster, c = other   ")
location_dict = {'a': "C:\\Users\\heine\\github\\expression_broad_data", 'b': "/home/heineike/github/expression_broad_data",'c':'you need to add your location to the location_dict'}
base_dir = location_dict[location_input]
print("base directory is " + base_dir)

if sys.path[-1] != base_dir:
    sys.path.append(base_dir)
    print("Added " + base_dir + " to path: " )
    print(sys.path)
import os

print("I am about to import a library")
from core import expression_plots 
from core import io_library 
%load_ext autoreload
%autoreload 2
# %matplotlib 
# plt.ioff()
#%matplotlib inline
%matplotlib notebook
data_processing_dir = base_dir + os.sep + os.path.normpath("expression_data") + os.sep

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="ticks", color_codes=True)
from sklearn import linear_model
import pickle
import subprocess
import networkx as nx

from Bio import SeqIO
from Bio import SeqFeature as sf
from Bio.Alphabet import generic_dna
from Bio.Seq import Seq

import re

# from collections import Counter
# import scipy.stats as stats
# from itertools import chain
#this only works if you are online
online_input = input("are you online? Yes/No")
if online_input == "Yes": 
    import plotly.plotly as py
    import plotly.graph_objs as pygo
    import plotly.tools as pytools
    py.sign_in('heineike02', 'APPjKrtARaN2ZgUYIkqr')

what computer are you on? a = Bens, b = gpucluster, c = other   b
base directory is /home/heineike/github/expression_broad_data
Added /home/heineike/github/expression_broad_data to path: 
['', '/usr/lib/python35.zip', '/usr/lib/python3.5', '/usr/lib/python3.5/plat-x86_64-linux-gnu', '/usr/lib/python3.5/lib-dynload', '/usr/local/lib/python3.5/dist-packages', '/usr/local/lib/python3.5/dist-packages/cmonkey2-1.2.6-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/SQLAlchemy_Utils-0.32.14-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/svgwrite-1.1.11-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/Routes-2.4.1-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/CherryPy-10.2.2-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/biopython-1.69-py3.5-linux-x86_64.egg', '/usr/local/lib/python3.5/dist-packages/configparser-3.5.0-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/rpy2-2.8.6-py3.5-linux-x86_64.egg', '/usr/local/lib/python3.5/dist-packages/repoze.lru-0.6-py3.5.egg', '

In [2]:
%load_ext autoreload
%autoreload 2
# %matplotlib 
# plt.ioff()
#%matplotlib inline
%matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Identify all paralogs
subset paralogs by whole genome duplication. 

In [3]:
#Total number of WGD paralogs
ohnologs = pd.read_csv(data_processing_dir + os.path.normpath("ortholog_files_YGOB/ohnologs.csv"))
print('Total number of Ohnologs: ' + str(len(ohnologs)))

Total number of Ohnologs: 547


In [4]:
#Total number of KL genes that have a paralog in SCER
fname_cross_spec_comparison = data_processing_dir + os.path.normpath("20170817_klscpka/cross_species_compare_ASminus_ASplus_YPD.csv")
kl_sc_PKA_data = pd.read_csv(fname_cross_spec_comparison, index_col=0)

kl_sc_PKA_data_paralogs = kl_sc_PKA_data[kl_sc_PKA_data["N SCer Orthologs"]==2]

print('Total number of KL genes that map to a set of paralogs in S.Cer: ' + str(int(len(kl_sc_PKA_data_paralogs)/2)))

#Total of these that are WGD paralogs
ohnolog_set = set(ohnologs['Gene 1']) | set(ohnologs['Gene 2'])

non_ohnologs = set(kl_sc_PKA_data_paralogs['sc_genename']) - ohnolog_set
ohnologs_klac_orth = set(kl_sc_PKA_data_paralogs['sc_genename']) & ohnolog_set

print('KL genes that map to a set of Ohnologs: ' + str(int(len(ohnologs_klac_orth)/2)) + '\n' +
     'KL genes that map to a non Ohnolog paralog: ' + str(int(len(non_ohnologs))))


Total number of KL genes that map to a set of paralogs in S.Cer: 519
KL genes that map to a set of Ohnologs: 519
KL genes that map to a non Ohnolog paralog: 0


Apparently the ortholog mapping I am using only includes paralogs from the WGH.  I am sure there must be others.  I should probably investigate this at some stage. 

In [120]:
#What happens under PKA inhibition for SCer Genes that don't have a K.Lactis ortholog? 

ohnologs_no_kl_orth = ohnolog_set-set(kl_sc_PKA_data_paralogs['sc_genename'])

#Doesn't work - pickle breaks between gpu and my computer
#YPD data
# fname_comparison_data = "20170817_klscpka/ASminus_ASplus_YPD_comparison.pkl"
# with open(data_processing_dir + os.path.normpath(fname_comparison_data), 'rb') as pkl_in:
#      comparison_data = pickle.load(pkl_in)

sc_ypd_foldchange = pd.read_csv(data_processing_dir + os.path.normpath('20170817_klscpka/20170817_klscpka_DESEQ_YPD_t40_AsMinvAsPlus_SC.csv'))
sc_avg_counts = pd.read_csv(data_processing_dir + os.path.normpath('20170817_klscpka/20170817_klscpka_avg_counts_SC.csv'))
sc_avg_counts.rename(columns={'0':'sc_genename'},inplace=True)

sc_ypd_data = sc_ypd_foldchange.rename(columns= {'Unnamed: 0':'sc_genename'})
sc_ypd_data.set_index('sc_genename', inplace = True)
sc_ypd_data = sc_ypd_data.assign(SC_common_name = io_library.SC_common_name_lookup(sc_ypd_data.index))

sc_ypd_data = sc_avg_counts.join(sc_ypd_data, on='sc_genename')
sc_ypd_data.set_index('sc_genename', inplace = True)
sc_ypd_data


,AS_noNMPP1_SDC_t40_mean,AS_noNMPP1_SDC_t40_var,AS_yesNMPP1_SDC_t40_mean,AS_yesNMPP1_SDC_t40_var,WT_noNMPP1_SDC_t40_mean,WT_noNMPP1_SDC_t40_var,WT_yesNMPP1_SDC_t40_mean,WT_yesNMPP1_SDC_t40_var,AS_noNMPP1_YPD_t40_mean,AS_noNMPP1_YPD_t40_var,...,AS_noNMPP1_SDC_t20_var,AS_yesNMPP1_SDC_t20_mean,AS_yesNMPP1_SDC_t20_var,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,SC_common_name
sc_genename,,,,,,,,,,,,,,,,,,,,,
YAL012W,3.413158,0.014747,3.337415,0.008049,3.352534,0.013363,3.252958,0.000013,3.408142,0.000657,...,0.004576,3.370129,0.010604,3164.003504,-0.441366,0.278283,-1.586033,1.127318e-01,2.004965e-01,CYS3
YAL069W,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,YAL069W
YAL068W-A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.121944,0.029740,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,YAL068W-A
YAL068C,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.124758,0.046693,0.000000,NaN,NaN,NaN,NaN,NaN,PAU8
YAL067W-A,0.000000,0.000000,0.142101,0.060578,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.144697,0.062812,0.145147,1.379840,4.080473,0.338157,7.352450e-01,8.303820e-01,YAL067W-A
YAL067C,0.352827,0.098438,0.709097,0.411962,1.129172,0.016701,1.179267,0.004345,0.572047,0.006683,...,0.058953,0.894348,0.050347,5.260742,0.754235,0.989719,0.762070,4.460184e-01,5.963246e-01,SEO1
YAL066W,0.000000,0.000000,0.142101,0.060578,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.124758,0.046693,0.200705,1.379840,4.080473,0.338157,7.352450e-01,8.303820e-01,YAL066W
YAL065C,0.193503,0.112330,0.000000,0.000000,0.078784,0.018621,0.327041,0.013829,0.164679,0.081357,...,0.077942,0.144697,0.062812,0.686532,-1.104253,3.055574,-0.361390,7.178081e-01,8.282432e-01,YAL065C
YAL064W-B,0.000000,0.000000,0.000000,0.000000,0.078784,0.018621,0.000000,0.000000,0.061998,0.011531,...,0.000000,0.089721,0.024150,0.484299,1.205730,3.003989,0.401376,6.881432e-01,8.085897e-01,YAL064W-B


In [144]:
sc_ypd_data[sc_ypd_data['SC_common_name'].isin(['MSN2','MSN4'])][['AS_noNMPP1_YPD_t40_mean','AS_yesNMPP1_YPD_t40_mean']]

,AS_noNMPP1_YPD_t40_mean,AS_yesNMPP1_YPD_t40_mean
sc_genename,,
YKL062W,0.932825,1.448539
YMR037C,0.878076,1.113982


In [121]:
ohnologs_no_kl_orth_pairs = ohnologs[ohnologs['Gene 1'].isin(ohnologs_no_kl_orth)]

print('Pairs of paralogs in SC with no kl ortholog: ' + str(len(ohnologs_no_kl_orth_pairs)))

#Go through this and assign fold change to each one.  

new_data = {'gene1_LFC': [], 'gene1_avg_exp_noNMPP1': [],'gene1_avg_exp_yesNMPP1': [],
            'gene2_LFC': [], 'gene2_avg_exp_noNMPP1': [],'gene2_avg_exp_yesNMPP1': []}

for ind, row in ohnologs_no_kl_orth_pairs.iterrows(): 
    gene1 = row['Gene 1']
    new_data['gene1_LFC'].append(float(sc_ypd_data.loc[gene1]['log2FoldChange']))
    new_data['gene1_avg_exp_noNMPP1'].append(float(sc_ypd_data.loc[gene1]['AS_noNMPP1_YPD_t40_mean']))
    new_data['gene1_avg_exp_yesNMPP1'].append(float(sc_ypd_data.loc[gene1]['AS_yesNMPP1_YPD_t40_mean']))
    
    gene2 = row['Gene 2']
    new_data['gene2_LFC'].append(float(sc_ypd_data.loc[gene2]['log2FoldChange']))
    new_data['gene2_avg_exp_noNMPP1'].append(float(sc_ypd_data.loc[gene2]['AS_noNMPP1_YPD_t40_mean']))
    new_data['gene2_avg_exp_yesNMPP1'].append(float(sc_ypd_data.loc[gene2]['AS_yesNMPP1_YPD_t40_mean']))

ohnologs_no_kl_orth_pairs.assign(**new_data)[['Gene Name 1','gene1_LFC','gene1_avg_exp_noNMPP1','gene1_avg_exp_yesNMPP1','Gene Name 2', 'gene2_LFC','gene2_avg_exp_noNMPP1','gene2_avg_exp_yesNMPP1']].sort_values(by='gene1_LFC')

# for ind, column in enumerate(columns): 
#     a[column] = [row[ind] for row in b]

Pairs of paralogs in SC with no kl ortholog: 28


,Gene Name 1,gene1_LFC,gene1_avg_exp_noNMPP1,gene1_avg_exp_yesNMPP1,Gene Name 2,gene2_LFC,gene2_avg_exp_noNMPP1,gene2_avg_exp_yesNMPP1
520,SPO19,-4.840507,0.713965,0.000000,YOR214C,2.420921,0.249763,0.837610
534,RPL20A,-3.936965,3.412258,2.367661,RPL20B,-4.111787,3.301923,2.202816
543,ATR1,-2.128023,2.374841,1.862896,YMR279C,-1.493332,0.720556,0.498756
435,RPL41A,-2.036833,3.657479,3.178033,RPL41B,-1.805427,3.035567,2.703081
417,TIR3,-1.511669,2.460936,2.132716,TIR2,0.264392,1.615583,1.816379
215,MIG3,-1.339809,1.399376,1.173717,MIG2,-2.603948,1.995060,1.357922
190,BAP2,-1.192563,2.766405,2.572179,BAP3,-1.564176,2.917730,2.581011
175,YDL012C,-0.955096,2.531284,2.382145,YBR016W,1.006656,2.482495,2.920191
293,RSC3,-0.903158,1.856831,1.758987,RSC30,0.186450,0.735845,0.823483
31,LCB3,-0.753349,2.341170,2.248597,YSR3,0.434324,1.766047,2.025983


HXT1 and 6.  6 low expression - expressed a bit more.  1 high expression - decreased significantly. 
YPR1 and GCY1:  GCY1 highly expressed, YPR 1 induced a bit. 
HXT7/HXT4: 7 up, 4 down.  
YOR186W/YLR297W: 186W up 297W up a bit. 297W goes from nucleus to the vacuole underdna rep stress.  186W localized to vacuole.  
SPO19: one up one down, but not very highly expressed. Sporulation

both: 
GPX1/HYR1: GPX - oxidative stress, HYR1 oxidative stress sensor
HOR7/DDR2: DDR2 very highly expressed under stress.  function really not known (SGD)
PDR5/PDR15: ABC transporter
YBR284W/YJL070C: function not clear. 
SPS100/YGP1: SPS100 goes from barely expressed to high.  YGP1 already relatively highly expressed gets higher. Cell wall proteins.  SPS100 sporulation related. 

All jointly repressed: 
534	RPL20A	-3.936965	3.412258	2.367661	RPL20B	-4.111787	3.301923	2.202816
543	ATR1	-2.128023	2.374841	1.862896	YMR279C	-1.493332	0.720556	0.498756
435	RPL41A	-2.036833	3.657479	3.178033	RPL41B	-1.805427	3.035567	2.703081
215	MIG3	-1.339809	1.399376	1.173717	MIG2	-2.603948	1.995060	1.357922
190	BAP2	-1.192563	2.766405	2.572179	BAP3	-1.564176	2.917730	2.581011

Mig2/Mig3 cause glucose repression.  
ATR1 is a drug pump - SGD says it increases under stress.  Paralog might be a boron transporter- also should be induced (SGD)  but doesn't seem to be here. 
BAP2/3: amino acid uptake (really not decreased that much) 

In [141]:
#For SC genes that have no KL orthologs, what genes change drastically under PKA inhibition?  
#Already looked at paralogs above. 

sc_no_kl_orths = set(sc_avg_counts['sc_genename'])-set(kl_sc_PKA_data['sc_genename'])

sc_no_kl_orths_no_paralogs = sc_no_kl_orths - (set(ohnologs_no_kl_orth_pairs['Gene 1']) | set(ohnologs_no_kl_orth_pairs['Gene 2']))           
                                               
print("There are {} sc genes with no kl ortholog.  Of those {} do not have ohnologs".format(len(sc_no_kl_orths), len(sc_no_kl_orths_no_paralogs)))

columns_to_view = ['SC_common_name', 'log2FoldChange', 'AS_noNMPP1_YPD_t40_mean', 'AS_yesNMPP1_YPD_t40_mean']

sc_ypd_data_sc_only = sc_ypd_data.loc[sc_no_kl_orths]

sc_ypd_data_sc_only_pka_change = sc_ypd_data_sc_only[abs(sc_ypd_data_sc_only['log2FoldChange'])>2.0]

print("Of the {} sc genes with no kl ortholog, {} of them have fold change greater than 2.0 (up or down) under PKA inhibition.  20 of those are paralogs".format(
        len(sc_no_kl_orths), len(sc_ypd_data_sc_only_pka_change)))

sc_ypd_data_sc_only_pka_change.sort_values(by='log2FoldChange')[columns_to_view]



There are 1724 sc genes with no kl ortholog.  Of those 1668 do not have ohnologs
Of the 1724 sc genes with no kl ortholog, 336 of them have fold change greater than 2.0 (up or down) under PKA inhibition.  20 of those are paralogs


,SC_common_name,log2FoldChange,AS_noNMPP1_YPD_t40_mean,AS_yesNMPP1_YPD_t40_mean
sc_genename,,,,
YOR309C,YOR309C,-6.754461,2.443750,0.637542
YGL168W,HUR1,-6.738577,1.198944,0.000000
YBR092C,PHO3,-5.975478,2.395164,0.832014
YDL039C,PRM7,-5.844456,2.760269,1.187281
YDL227C,HO,-5.497170,2.999891,1.497682
YBR191W-A,YBR191W-A,-5.068049,1.881674,0.954339
YDR281C,PHM6,-4.843624,1.742183,0.546998
YPL130W,SPO19,-4.840507,0.713965,0.000000
YMR042W,ARG80,-4.802050,0.594620,0.000000


PAI3: Proteinase A inhibitor
GPG1: gamma subunit interacting with GPR1
RRT1: unknown function - deletion effect ribosomal dna transcription
AMS1: Oligosaccharide degradation.  
DIA3: pseudohyphal growth
HSP31: converts methylglyoxal to D-lactate;
PDC6: amino acid catabolism

Probably just overlapping HXK1:
YFR052C-A	YFR052C-A	4.902003	2.239221	3.865268
Probably just overlapping SYM1 (mitochondrial protein)
YLR252W	YLR252W	4.602415	1.455752	2.954681

Probably just overlapping PMR1
YGL168W	HUR1	-6.738577	1.198944	0.000000

# Subsets comparing K.Lac and S.Cer expression for differentially induced paralogs. 

,Gene 1,Gene Name 1,kl_genename_gene1,Gene 2,Gene Name 2,kl_genename_gene2,log2FoldChange_SC_gene1,log2FoldChange_SC_gene2,log2FoldChange_KL_gene1
0,YCL051W,LRE1,KLLA0C00803g,YDR528W,HLR1,KLLA0C00803g,1.006787,-3.262097,-0.749493
1,YCL050C,APA1,KLLA0C00847g,YDR530C,APA2,KLLA0C00847g,-0.184330,1.719713,0.725481
2,YDR524C-B,YDR524C-B,KLLA0C00957g,YCL048W-A,YCL048W-A,KLLA0C00957g,-2.236981,7.830939,-0.149862
3,YCL048W,SPS22,KLLA0C01001g,YDR522C,SPS2,KLLA0C01001g,2.212668,-1.076672,2.658001
4,YCL043C,PDI1,KLLA0C01111g,YDR518W,EUG1,KLLA0C01111g,-0.003622,-0.030605,1.039729
5,YCL040W,GLK1,KLLA0C01155g,YDR516C,EMI2,KLLA0C01155g,4.160390,2.464130,2.178913
6,YCL037C,SRO9,KLLA0C01210g,YDR515W,SLF1,KLLA0C01210g,-4.754452,0.379591,-4.762072
7,YCL036W,GFD2,KLLA0C01254g,YDR514C,YDR514C,KLLA0C01254g,-3.913143,-2.131266,-0.748394
8,YCL035C,GRX1,KLLA0C01298g,YDR513W,GRX2,KLLA0C01298g,2.965390,3.051427,1.719177
9,YCL025C,AGP1,KLLA0C01606g,YDR508C,GNP1,KLLA0C01606g,0.734168,-2.082446,-2.184941


In [5]:
#build a dataframe that has all SC paralogs paired up with log fold change and mean expression columns.  Also has K.Lactis log fold change and mean expression columns. 

ohnologs_expression_gene1 = pd.merge(ohnologs, kl_sc_PKA_data_paralogs, how = 'inner', left_on = 'Gene 1', right_on = 'sc_genename')
ohnologs_expression = pd.merge(ohnologs_expression_gene1, kl_sc_PKA_data_paralogs, how = 'inner', left_on = 'Gene 2', right_on = 'sc_genename', suffixes = ['_gene1', '_gene2'])



#Assign sc threshold groups
high_threshold_SC = 2.0
low_threshold_SC = -high_threshold_SC
ohnologs_expression = ohnologs_expression.assign(threshold_group_SC = lambda x: io_library.threshold_group_SC_series(x.log2FoldChange_SC_gene1,x.log2FoldChange_SC_gene2,high_threshold_SC, low_threshold_SC))

#Assign kl threshold groups
high_threshold_KL = high_threshold_SC
low_threshold_KL = low_threshold_SC
ohnologs_expression = ohnologs_expression.assign(threshold_group_KL = lambda x: io_library.threshold_group_KL_series(x.log2FoldChange_KL_gene1, high_threshold_KL, low_threshold_KL))



#filter to genes with separated expression
foldchange_separation = 3.5

ohnologs_expression_separated = ohnologs_expression[abs(ohnologs_expression['log2FoldChange_SC_gene1'] - ohnologs_expression['log2FoldChange_SC_gene2']) > foldchange_separation]

columns_to_view = ['Gene 1', 'Gene Name 1', 'kl_genename_gene1', 'Gene 2', 'Gene Name 2', 'kl_genename_gene2', 'log2FoldChange_SC_gene1', 'log2FoldChange_SC_gene2', 'threshold_group_SC','log2FoldChange_KL_gene1','threshold_group_KL']

ohnologs_expression_separated[columns_to_view]




,Gene 1,Gene Name 1,kl_genename_gene1,Gene 2,Gene Name 2,kl_genename_gene2,log2FoldChange_SC_gene1,log2FoldChange_SC_gene2,threshold_group_SC,log2FoldChange_KL_gene1,threshold_group_KL
0,YCL051W,LRE1,KLLA0C00803g,YDR528W,HLR1,KLLA0C00803g,1.006787,-3.262097,down_flat,-0.749493,flat
2,YDR524C-B,YDR524C-B,KLLA0C00957g,YCL048W-A,YCL048W-A,KLLA0C00957g,-2.236981,7.830939,up_down,-0.149862,flat
6,YCL037C,SRO9,KLLA0C01210g,YDR515W,SLF1,KLLA0C01210g,-4.754452,0.379591,down_flat,-4.762072,down
19,YCR034W,FEN1,KLLA0C03542g,YJL196C,ELO1,KLLA0C03542g,-2.858003,1.016717,down_flat,-0.688902,flat
24,YJL164C,TPK1,KLLA0B07205g,YKL166C,TPK3,KLLA0B07205g,2.844732,-2.613249,up_down,0.774367,flat
26,YJL159W,HSP150,KLLA0B07392g,YKL163W,PIR3,KLLA0B07392g,0.711945,6.987255,up_flat,0.254423,flat
35,YJL110C,GZF3,KLLA0F21296g,YKR034W,DAL80,KLLA0F21296g,-1.197971,2.719467,up_flat,-0.648261,flat
42,YJL079C,PRY1,KLLA0D02420g,YKR013W,PRY2,KLLA0D02420g,1.883493,-2.156467,down_flat,4.629306,up
50,YFR017C,IGD1,KLLA0A10967g,YOL024W,YOL024W,KLLA0A10967g,5.532256,1.159039,up_flat,2.714056,up
56,YLR249W,YEF3,KLLA0F12210g,YNL014W,HEF3,KLLA0F12210g,-5.379224,3.247235,up_down,-3.102575,down


In [270]:
#Bar chart showing expression groups in separated and not separated sets. 

expression_groups = ['flat_flat', 'up_flat','down_down','down_flat', 'up_up','up_down']

data_sets = {'All Paralogs': ohnologs_expression, 'Separated Paralogs': ohnologs_expression_separated}

# data_sets['Separated Paralogs']['threshold_group'].value_counts()[expression_groups]

traces = []

for name, data_set in data_sets.items():

    counts = data_set['threshold_group_SC'].value_counts()[expression_groups].fillna(0)
    total_counts = sum(counts)
    percentages = counts/total_counts*100
    
    trace = pygo.Bar(
        x=expression_groups,
        y=percentages,
        text=['{} <br> {:2.0f}%'.format(item, item/total_counts*100) for item in list(counts)],
        textposition = 'auto',
        insidetextfont = {'color': 'white'},
        name = name
    #     marker=dict(
    #             color='rgb(158,202,225)',
    #             line=dict(
    #                 color='rgb(8,48,107)',
    #                 width=1.5),
    #             ),
    #     opacity=0.6
        )
    
    traces.append(trace)

data = traces
layout = pygo.Layout(
    barmode='group'
)

fig = pygo.Figure(data=data, layout=layout)
py.iplot(fig, filename='paralog_expression_groups')




In [281]:
#Bar chart showing KL expressionin separated genes by group

expression_groups = ['flat_flat', 'up_flat','down_down','down_flat', 'up_up','up_down']
kl_foldchange_groups = ['up', 'flat', 'down']

#build dataframe with counts for each level

data_set = ohnologs_expression #ohnologs_expression_separated

count_data = {}
for group in kl_foldchange_groups: 

    kl_group = data_set[data_set['threshold_group_KL']==group]

    counts = kl_group['threshold_group_SC'].value_counts()[expression_groups].fillna(0)

    count_data[group] = counts

count_data_df = pd.DataFrame(count_data, index = expression_groups)

traces = []

for group in kl_foldchange_groups:

    counts = count_data_df[group]
    #total_counts = sum(counts)
    #percentages = counts/total_counts*100
    
    trace = pygo.Bar(
        x=expression_groups,
        y=counts,
        text= counts,
        textposition = 'auto',
        insidetextfont = {'color': 'white'},
        name = 'kl exp ' + group
    #     marker=dict(
    #             color='rgb(158,202,225)',
    #             line=dict(
    #                 color='rgb(8,48,107)',
    #                 width=1.5),
    #             ),
    #     opacity=0.6
        )
    
    traces.append(trace)

data = traces
layout = pygo.Layout(
    barmode='stack',
    xaxis = {'title': 'SCer paralog expression groups'},
    yaxis = {'title': 'Number of paralog pairs'}
)

fig = pygo.Figure(data=data, layout=layout)
py.iplot(fig, filename='paralog_expression_groups_kl_exp')


In [197]:
AB

,A,B
0,-2.5,2.2
1,1.0,4.1


In [203]:




df = pd.DataFrame([[-2.5,2.2], [1.0,4.1]], columns =['A','B'] )
df

#io_library.threshold_group_series(A,B, high_threshold, low_threshold)

# io_library.threshold_group(b,a,high_threshold, low_threshold)

# #input: a, b, high_threshold, low_threshold
# #output: group - either {'up_up', 'up_flat','up_down','flat_flat', 'down_flat','down_down'}

# a_sign = io_library.threshold_sign(a, high_threshold, low_threshold)
# b_sign = io_library.threshold_sign(b, high_threshold, low_threshold)

# sign_sum = a_sign+b_sign

# if sign_sum ==2:
#     group = 'up_up'
# elif sign_sum == -2: 
#     group = 'down_down'
# elif sign_sum == 1: 
#     group = 'up_flat'
# elif sign_sum == -1: 
#     group = 'down_flat'
# elif sign_sum == 0:
#     if a_sign-b_sign == 0:
#         group = 'flat_flat'
#     else: 
#         group = 'up_down'
# else: 
#     print('Error - did not assign group')

# return group



,A,B,new
0,-2.5,2.2,up_down
1,1.0,4.1,up_flat


In [ ]:
#bar chart

x = ['Product A', 'Product B', 'Product C']
y = [20, 14, 23]
y2 = [16,12,27]

trace1 = go.Bar(
    x=x,
    y=y,
    text=y,
    textposition = 'auto',
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5),
        ),
    opacity=0.6
)

trace2 = go.Bar(
    x=x,
    y=y2,
    text=y2,
    textposition = 'auto',
    marker=dict(
        color='rgb(58,200,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5),
        ),
    opacity=0.6
)

data = [trace1,trace2]

py.iplot(data, filename='grouped-bar-direct-labels')




In [19]:
ohnologs_expression[ohnologs_expression['Gene Name 1']=='ADH5'][columns_to_view]



,Gene 1,Gene Name 1,kl_genename_gene1,Gene 2,Gene Name 2,kl_genename_gene2,log2FoldChange_SC_gene1,log2FoldChange_SC_gene2,threshold_group_SC,log2FoldChange_KL_gene1,threshold_group_KL
163,YBR145W,ADH5,KLLA0F21010g,YOL086C,ADH1,KLLA0F21010g,0.148592,0.099784,flat_flat,0.096766,flat
